# Victor PAPIN - Data & IA TD 2 - TP 3


Dataset : https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

My kaggle : https://www.kaggle.com/viwu99


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re

from numpy.core.numeric import normalize_axis_tuple
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import seaborn as sns

data = pd.read_csv('twitter_training.csv',header=None, names=["tweet_id", "entity", "sentiment", "text"])

data.head()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   74682 non-null  int64 
 1   entity     74682 non-null  object
 2   sentiment  74682 non-null  object
 3   text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [2]:
data = data.drop(columns=['tweet_id', 'entity'])


In [3]:
import re
def normalize(data): #from the example
    normalized = []
    for i in data:
        i = str(i).lower()
        # get rid of urls

        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
#identifying the feature // target
features = data['text']
targets = encoder.fit_transform(data['sentiment'])


X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.20, random_state=18)
print(X_train)

# normalise

X_train = normalize(X_train)
X_test = normalize(X_test)

# configuring tokenizer

max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

# tokenize

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)




53712    Happy 4th of July to the @callofduty nation. E...
50974    I ’ m unstoppable compared with Bayern FC Muni...
10985    @EricGHarrison that would so get the Halo char...
39855                         Fair play I’m pretty high on
32155                      Grinding.. twitch.tv / krrnster
                               ...                        
70830    The CQC takedowns are some of my favourite in ...
49073    mfs be 17 in a training facilities at fifa, br...
1726     Team My sona as a vault hunter. Borderlands wi...
35653    I have never been prouder (or more grateful) t...
60714        gta 5 rp is trash no one wanted to help me . 
Name: text, Length: 59745, dtype: object


In [4]:
#Making the RNN (here LSTM)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_vocab, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4)
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
data.head(50)

,sentiment,text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
5,Positive,im getting into borderlands and i can murder y...
6,Positive,So I spent a few hours making something for fu...
7,Positive,So I spent a couple of hours doing something f...
8,Positive,So I spent a few hours doing something for fun...
9,Positive,So I spent a few hours making something for fu...


In [8]:
#stop the training
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, padding="post", maxlen=256)
X_test  = pad_sequences(X_test, padding="post", maxlen=256)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(3 * 1e-4), # 3 * 1e-4 because 1e-4 was too slow
              metrics=['accuracy'])

# For the part 5 of the exercice, I tried this because i was stuck at a 0.8714 accuracy
def scheduler(epoch, lr):
    return lr * 0.7

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)


history = model.fit(X_train, y_train, epochs=10,validation_split=0.1, batch_size=30, shuffle=True, callbacks=[early_stop,lr_scheduler])

Epoch 1/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 180s 99ms/step - accuracy: 0.9036 - loss: 0.2665 - val_accuracy: 0.8373 - val_loss: 0.4856 - learning_rate: 2.1000e-04
Epoch 2/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 161s 90ms/step - accuracy: 0.9185 - loss: 0.2226 - val_accuracy: 0.8403 - val_loss: 0.5108 - learning_rate: 1.4700e-04
Epoch 3/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.9268 - loss: 0.1983 - val_accuracy: 0.8445 - val_loss: 0.5416 - learning_rate: 1.0290e-04


It stopped at accuracy: 0.9268

In [10]:
# evaluating

model.evaluate(X_test, y_test)

pred = model.predict(X_test)



467/467 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.8336 - loss: 0.5068
467/467 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step


And so i end up with an accuracy of 0.83 wich is really okey because that means a 83% of good respond wich I'm okey with